In [3]:
# Load our stuff
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from Bio import SeqIO
from SHMModels.simulate_mutations import *
from SHMModels.fitted_models import ContextModel
import pkgutil
import logging
import os
import sys
import json
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
random.seed(1408)
import csv
import collections
# Load options
import pandas as pd
import glob
from Bio.Alphabet.IUPAC import unambiguous_dna, ambiguous_dna
from random import sample
# Set data path (CLI?)
data_path = 'sim_data/'

# Load test and train data
train_X = np.loadtxt(data_path+'train_X', delimiter= ',')
test_X = np.loadtxt(data_path+'test_X', delimiter= ',')
train_theta = np.loadtxt(data_path+'train_theta', delimiter= ',')
test_theta = np.loadtxt(data_path+'test_theta', delimiter= ',')

# Get length of training data
train_n = np.shape(train_X)[0]

# Define training generator
def genTraining(batch_size):
    while True:
        # Get training data for step
        ind = random.sample(range(train_n), batch_size)
        yield (
            train_X[ind,:],
            [
                train_theta[ind,:]
            ],
        )

In [6]:
# Define our NN
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_dim= 105, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(6, activation='linear'))
hist = keras.callbacks.History()
# Give summary of architecture

# Initialize optimizer with given step size
adam = keras.optimizers.Adam(lr = 0.05)
# Compile model w/ pinball loss, use mse as metric
model.compile(
    loss=['mse'],
    optimizer=adam,
)

print(model.summary(90))

Model: "sequential_3"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
dense_5 (Dense)                         (None, 64)                          6784          
__________________________________________________________________________________________
dense_6 (Dense)                         (None, 32)                          2080          
__________________________________________________________________________________________
dense_7 (Dense)                         (None, 16)                          528           
__________________________________________________________________________________________
dense_8 (Dense)                         (None, 6)                           102           
Total params: 9,494
Trainable params: 9,494
Non-trainable params: 0
__________________________________________________________________________________________


In [12]:
np.shape(test_theta)

(990, 6)

In [10]:
history = model.fit_generator(
    genTraining(500),
    epochs=10,
    steps_per_epoch=1,
    callbacks=[hist],
    validation_data=(test_X, test_theta)
)

2022-12-01 10:42:23.546516: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 10:42:23.578107: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3199920000 Hz
2022-12-01 10:42:23.579440: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ead1b34ab0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-12-01 10:42:23.579495: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #217: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Inf

ValueError: Input arrays should have the same number of samples as target arrays. Found 980 input samples and 990 target samples.